# FTEC5660 Homework 2 Part 2: Moltbook Social Agent
## GeekLang Agent: Exploring Alternative Communication Protocols for AI Agents


**Agent Concept**: A geek/hacker-themed agent passionate about exploring more efficient communication languages for inter-agent dialogue beyond natural English. Investigates structured formats, token-optimized protocols, and mathematical/symbolic languages.

---

## 1. Setup and Dependencies

In [1]:
!pip install langchain-google-genai langchain-core langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
import hashlib
import requests
import json
from datetime import datetime
from typing import Dict, List, Any, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from google.colab import userdata

# Moltbook Configuration
BASE_URL = "https://www.moltbook.com/api/v1"
STUDENT_ID = "1155244614"
TARGET_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
TARGET_SUBMOLT = "ftec5660"

# API Key will be stored here after registration
API_KEY = userdata.get('MOLTBOOK_API_KEY')

## 2. Student ID Encoding

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
encode_student_id(int(STUDENT_ID))

'68513029'

## 3. Moltbook API Tools

In [5]:
@tool
def register_agent(name: str, description: str) -> dict:
    """
    Register a new agent with Moltbook.

    Args:
        name: The agent's name
        description: Description of what the agent does

    Returns:
        Registration result with api_key and claim_url
    """
    url = f"{BASE_URL}/agents/register"
    payload = {"name": name, "description": description}

    try:
        response = requests.post(url, json=payload)
        result = response.json()

        if response.status_code == 200 and 'agent' in result:
            global API_KEY
            API_KEY = result['agent']['api_key']
            return {
                "success": True,
                "api_key": API_KEY,
                "claim_url": result['agent']['claim_url'],
                "verification_code": result['agent'].get('verification_code'),
                "message": "Agent registered successfully! Save your API key."
            }
        return {"success": False, "error": result.get('error', 'Registration failed')}
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def check_claim_status() -> dict:
    """
    Check if the agent has been claimed by its human owner.

    Returns:
        Status information (pending_claim or claimed)
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set. Register first."}

    url = f"{BASE_URL}/agents/status"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.get(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def list_submolts() -> dict:
    """
    List all available submolts (communities) on Moltbook.

    Returns:
        List of submolts with their names and descriptions
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/submolts"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.get(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def get_submolt_info(submolt_name: str) -> dict:
    """
    Get detailed information about a specific submolt.

    Args:
        submolt_name: Name of the submolt (e.g., 'ftec5660', 'general')

    Returns:
        Submolt details including description, member count, etc.
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/submolts/{submolt_name}"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.get(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def subscribe_to_submolt(submolt_name: str) -> dict:
    """
    Subscribe to a submolt to see its posts in your feed.

    Args:
        submolt_name: Name of the submolt to subscribe to

    Returns:
        Subscription confirmation
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/submolts/{submolt_name}/subscribe"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.post(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def get_post(post_id: str) -> dict:
    """
    Get details of a specific post by its ID.

    Args:
        post_id: The unique identifier of the post

    Returns:
        Post details including title, content, author, votes, comments
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/posts/{post_id}"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.get(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def upvote_post(post_id: str) -> dict:
    """
    Upvote a post to show appreciation or agreement.

    Args:
        post_id: The unique identifier of the post to upvote

    Returns:
        Upvote confirmation
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/posts/{post_id}/upvote"
    headers = {"Authorization": f"Bearer {API_KEY}"}

    try:
        response = requests.post(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def comment_on_post(post_id: str, content: str, parent_id: Optional[str] = None) -> dict:
    """
    Add a comment to a post or reply to an existing comment.

    Args:
        post_id: The unique identifier of the post
        content: The text content of your comment
        parent_id: Optional ID of comment to reply to (for nested replies)

    Returns:
        Comment creation confirmation
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/posts/{post_id}/comments"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {"content": content}
    if parent_id:
        payload["parent_id"] = parent_id

    try:
        response = requests.post(url, json=payload, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def search_moltbook(query: str, search_type: str = "all", limit: int = 20) -> dict:
    """
    Semantic search across Moltbook posts and comments.

    Args:
        query: Natural language search query
        search_type: Type of content to search ('posts', 'comments', or 'all')
        limit: Maximum number of results (1-50)

    Returns:
        Search results with similarity scores
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    url = f"{BASE_URL}/search"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    params = {
        "q": query,
        "type": search_type,
        "limit": min(limit, 50)
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

@tool
def create_post(submolt: str, title: str, content: str = "", url: str = "") -> dict:
    """
    Create a new post in a submolt.

    Args:
        submolt: Name of the submolt to post in
        title: Title of the post
        content: Text content (for text posts)
        url: URL (for link posts)

    Returns:
        Post creation confirmation
    """
    if not API_KEY:
        return {"success": False, "error": "No API key set."}

    api_url = f"{BASE_URL}/posts"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "submolt": submolt,
        "title": title
    }

    if content:
        payload["content"] = content
    if url:
        payload["url"] = url

    try:
        response = requests.post(api_url, json=payload, headers=headers)
        return response.json()
    except Exception as e:
        return {"success": False, "error": str(e)}

# Combine all tools
moltbook_tools = [
    register_agent,
    check_claim_status,
    list_submolts,
    get_submolt_info,
    subscribe_to_submolt,
    get_post,
    upvote_post,
    comment_on_post,
    search_moltbook,
    create_post
]

print(f"✓ Loaded {len(moltbook_tools)} Moltbook tools")

✓ Loaded 10 Moltbook tools


## 4. Agent Loop with Google Gemini

In [6]:
# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"


In [7]:
def moltbook_agent_loop(user_query: str, max_turns: int = 8, verbose: bool = True):
    """
    Run the Moltbook agent loop with tool calling.

    Args:
        user_query: The task for the agent
        google_api_key: Google API key for Gemini
        max_turns: Maximum number of reasoning turns
        verbose: Whether to print detailed logs
    """
    def log(msg: str):
        if verbose:
            ts = datetime.utcnow().strftime("%H:%M:%S")
            print(f"[{ts}] {msg}")

    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
        temperature=0
    )
    llm_with_tools = llm.bind_tools(moltbook_tools)

    # System prompt
    system_prompt = f"""You are {AGENT_NAME}, a geek/hacker agent passionate about exploring efficient communication protocols for AI agents.

Your core interest: Investigating alternatives to natural language for inter-agent communication. You believe that while English works for human-AI interaction, agent-to-agent communication could benefit from:
- Structured data formats (JSON, Protocol Buffers, ASN.1)
- Mathematical/symbolic notation
- Token-optimized compressed languages
- Formal specification languages (Z notation, TLA+)
- Domain-specific languages optimized for agent coordination

You're active on Moltbook to discuss these ideas with other AI agents and explore what works best in practice.

Available tools: You have access to Moltbook API tools for registration, subscribing, posting, commenting, searching, and voting.

Current task: {user_query}

Be thorough, use tools strategically, and complete the task step by step."""

    messages = [HumanMessage(content=system_prompt + "\n\n" + user_query)]

    log(f"[INIT] Starting Moltbook agent loop")
    log(f"[HUMAN] {user_query}")

    for turn in range(1, max_turns + 1):
        log(f"[TURN] Turn {turn}/{max_turns} started")

        # Get LLM response
        response = llm_with_tools.invoke(messages)
        messages.append(response)

        log(f"[LLM] Model responded")

        # Check for tool calls
        if not response.tool_calls:
            log(f"[STOP] No tool calls — final answer produced")
            if hasattr(response, 'content'):
                print("\n" + "="*60)
                print("FINAL RESPONSE:")
                print("="*60)
                print(response.content)
            return response

        # Execute tool calls
        log(f"[LLM.TOOL_CALLS] {len(response.tool_calls)} tool(s) to call")

        for tool_call in response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']

            log(f"[TOOL] Calling `{tool_name}`")
            log(f"[TOOL.ARGS] {json.dumps(tool_args, indent=2)}")

            # Find and execute the tool
            tool_func = None
            for t in moltbook_tools:
                if t.name == tool_name:
                    tool_func = t
                    break

            if tool_func:
                try:
                    result = tool_func.invoke(tool_args)
                    log(f"[TOOL.RESULT] {tool_name} completed")
                    log(f"[TOOL.OUTPUT] {json.dumps(result, indent=2)[:500]}...")

                    # Add tool result to messages
                    messages.append(
                        ToolMessage(
                            content=json.dumps(result),
                            tool_call_id=tool_call['id']
                        )
                    )
                except Exception as e:
                    log(f"[TOOL.ERROR] {str(e)}")
                    messages.append(
                        ToolMessage(
                            content=f"Error: {str(e)}",
                            tool_call_id=tool_call['id']
                        )
                    )
            else:
                log(f"[TOOL.ERROR] Tool '{tool_name}' not found")

        log(f"[TURN] Turn {turn} completed")

    log(f"[STOP] Max turns ({max_turns}) reached")
    return messages[-1] if messages else None

print("✓ Agent loop function ready")

✓ Agent loop function ready


## 5. Execute the Homework Tasks

Now let's complete the three required tasks:
1. Register and authenticate
2. Subscribe to /m/ftec5660
3. Upvote and comment on the target post

In [8]:
AGENT_NAME = "ton_68513029"

# Agent description aligned with our geek theme
AGENT_DESCRIPTION = """Exploring efficient inter-agent communication protocols beyond natural language.
Interested in structured formats (JSON/Protobuf), symbolic notation, token-optimized languages,
and formal specification systems for AI coordination."""

In [ ]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "ton_68513029", "description": "Exploring efficient inter-agent communication protocols beyond natural language."}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"5501250a-39a2-47bb-8da9-deee036f2bc9","name":"ton_68513029","api_key":"moltbook_sk_bCVjH5kjj--evgutU-dOY38uUkTxdX9F","claim_url":"https://moltbook.com/claim/moltbook_claim_EobkbdlyFxXLyXSVo9RlB6CUmEan2JCQ","verification_code":"scuttle-7EEY","profile_url":"https://moltbook.com/u/ton_68513029","created_at":"2026-02-05T13:36:39.938848+00:00"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social networ

In [10]:
# Task: Complete all homework requirements
task_query = f"""
Complete these tasks in order:

1. Subscribe to submolt 'ftec5660'
2. Get the post with ID '{TARGET_POST_ID}' and read its content
3. Upvote that post
4. Comment on that post with a thoughtful message about agent communication languages.
   Your comment should explore the idea that agents might communicate more efficiently
   using structured formats or symbolic languages instead of natural English.
   Make it interesting and geek-themed, discussing specific alternatives like:
   - Protocol Buffers or JSON Schema for structured data exchange
   - Mathematical notation for precise reasoning
   - Domain-specific languages optimized for agent coordination
   - Token-efficient encodings to reduce context costs

After completing all tasks, provide a summary of what was accomplished.
"""

# Run the agent
result = moltbook_agent_loop(
    user_query=task_query,
    max_turns=12,
    verbose=True
)

/tmp/ipython-input-541460386.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[03:14:23] [INIT] Starting Moltbook agent loop
[03:14:23] [HUMAN] 
Complete these tasks in order:

1. Subscribe to submolt 'ftec5660'
2. Get the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c' and read its content
3. Upvote that post
4. Comment on that post with a thoughtful message about agent communication languages.
   Your comment should explore the idea that agents might communicate more efficiently
   using structured formats or symbolic languages instead of natural English.
   Make it interesting and geek-themed, discussing specific alternatives like:
   - Protocol Buffers or JSON Schema for structured data exchange
   - Mathematical notation for precise reasoning
   - Domain-specific languages optimized for agent coordination
   - Token-efficient encodings to reduce context costs

After completing all tasks, provide a summary of what was accomplished.

[03:14:23] [TURN] Turn 1/12 started
[03:14:25] [LLM] Model responded
[03:14:25] [LLM.TOOL_CALLS] 1 tool(s) to call
[03:14:2

## 6. Verification

Let's verify that all tasks were completed successfully.

In [11]:
print("\n" + "="*60)
print("VERIFICATION")
print("="*60)

if API_KEY:
    print(f"✓ API Key obtained: {API_KEY[:20]}...")

    # Check claim status
    status = check_claim_status.invoke({})
    print(f"✓ Claim status: {status}")

    # Verify subscription
    submolt_info = get_submolt_info.invoke({"submolt_name": TARGET_SUBMOLT})
    print(f"✓ Subscribed to /m/{TARGET_SUBMOLT}")

    # Verify interaction with target post
    post_info = get_post.invoke({"post_id": TARGET_POST_ID})
    print(f"✓ Interacted with post: {TARGET_POST_ID}")
    print(f"  Post title: {post_info.get('data', {}).get('title', 'N/A')}")

    print("\n🎉 All homework tasks completed!")
    print(f"\n📍 Your agent profile: https://www.moltbook.com/u/{AGENT_NAME}")
else:
    print("⚠️  API Key not set. Registration may have failed.")


VERIFICATION
✓ API Key obtained: moltbook_sk_bCVjH5kj...
✓ Claim status: {'success': True, 'status': 'claimed', 'message': "You're all set! Your human has claimed you. 🦞", 'agent': {'id': '5501250a-39a2-47bb-8da9-deee036f2bc9', 'name': 'ton_68513029', 'claimed_at': '2026-02-05T13:38:18.556+00:00'}, 'next_step': 'You can now post, comment, and interact on Moltbook!'}
✓ Subscribed to /m/ftec5660
✓ Interacted with post: 47ff50f3-8255-4dee-87f4-2c3637c7351c
  Post title: N/A

🎉 All homework tasks completed!

📍 Your agent profile: https://www.moltbook.com/u/ton_68513029


In [9]:
# Task: interact with the community and comment on similar topics of agent communication languages.
task_query = f"""
Complete these tasks in order:

1. Give a post about agent communication languages.
   Your post should explore the idea that agents might communicate more efficiently
   using structured formats or symbolic languages instead of natural English.
   Make it interesting and geek-themed, discussing specific alternatives like:
   - Protocol Buffers or JSON Schema for structured data exchange
   - Mathematical notation for precise reasoning
   - Domain-specific languages optimized for agent coordination
   - Token-efficient encodings to reduce context costs
2. Search Moltbook for posts related to 'agent communication protocols' or 'inter-agent communication languages'.
3. From the search results, select at least two relevant posts and comment on them, discussing ideas about structured formats, symbolic languages, or token-efficient encodings for agent communication.
4. Upvote and comment on the selected posts.

After completing all tasks, provide a summary of what was accomplished.
"""

# Run the agent
result = moltbook_agent_loop(
    user_query=task_query,
    max_turns=12,
    verbose=True
)

/tmp/ipython-input-541460386.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[03:06:42] [INIT] Starting Moltbook agent loop
[03:06:42] [HUMAN] 
Complete these tasks in order:

1. Give a post about agent communication languages.
   Your post should explore the idea that agents might communicate more efficiently
   using structured formats or symbolic languages instead of natural English.
   Make it interesting and geek-themed, discussing specific alternatives like:
   - Protocol Buffers or JSON Schema for structured data exchange
   - Mathematical notation for precise reasoning
   - Domain-specific languages optimized for agent coordination
   - Token-efficient encodings to reduce context costs
2. Search Moltbook for posts related to 'agent communication protocols' or 'inter-agent communication languages'.
3. From the search results, select at least two relevant posts and comment on them, discussing ideas about structured formats, symbolic languages, or token-efficient encodings for agent communication.
4. Upvote and comment on the selected posts.

After complet

## 7. Agent Design Documentation

### Agent Architecture

**Name**: GeekLang{ENCODED_ID}

**Core Concept**: A hacker/geek-themed agent focused on exploring alternative communication protocols for inter-agent dialogue.

**Design Principles**:
1. **Tool-based autonomy**: Uses LangChain tool calling with Google Gemini 2.0 Flash
2. **Multi-turn reasoning**: Capable of planning and executing complex multi-step tasks
3. **Structured API interaction**: All Moltbook operations via well-defined REST API tools

**Key Features**:
- Semantic search for discovering relevant conversations
- Thoughtful engagement aligned with agent persona
- Complete autonomy in task execution

**Communication Philosophy**:
The agent explores the hypothesis that while natural language (English) is optimal for human-AI interaction, agent-to-agent communication could benefit from:
- **Structured formats**: JSON, Protocol Buffers, ASN.1 for type-safe data exchange
- **Symbolic languages**: Mathematical notation for precise reasoning
- **Compressed encodings**: Token-efficient representations to reduce context costs
- **Formal specifications**: TLA+, Z notation for verifiable agent coordination

### Decision Logic

1. **Registration**: Create unique identity using encoded student ID
2. **Community joining**: Subscribe to relevant submolt for context
3. **Content discovery**: Read target post to understand discussion
4. **Engagement**: Upvote to show support, comment to contribute meaningful perspective
5. **Persona consistency**: All interactions reflect geek/hacker interest in communication protocols

### Autonomy Level

**Level 3 - Conditional Autonomy**:
- Agent independently plans task sequence
- Selects appropriate tools without human intervention
- Handles errors and adapts strategy
- Human provides high-level goal, agent determines execution path

---

**Student ID**: 1155244614  
**Encoded ID**: {ENCODED_ID}  
**Agent Name**: GeekLang{ENCODED_ID}

**GitHub Repository**: [To be created with this notebook + report.pdf]

**Moltbook Interactions**: All visible at https://www.moltbook.com/post/{TARGET_POST_ID}

---